### Development of the PRS algorithm
Each node in a pathway has three attributes: Node_genes, Node_value (NV), Node_weight (NW)

#### 1.1. Node_genes
Calculated in the main script and passed as attribute 

#### 1.2. Node_value (NV)

In [ ]:
# Each node is assigned a value derived from expression data. The following values are assigned to the node: 0 if the 
# corresponding gene or genes are not expressed, 1 if they are expressed but remain unchanged (non-significant), or the
# maximum fold-change value if one or more of the mapped transcripts is above threshold.
def path_node_value(log_fc,fc,pvalues,log_fc_threshold=1.5,pvalue_threshold=0.05):
    expr_genes=[]
    for n in range(len(log_fc)):
        if(log_fc[n]<log_fc_threshold): # not expressed
            expr_genes.append(0)
        else: # expressed
            if(pvalues[n]>pvalue_threshold): # non-significant
                expr_genes.append(1)
            else: # significant
                expr_genes.append(np.max(fc))
    return expr_genes

#### 1.3. Node_weight (NW)

In [ ]:
# Generate each sub-path in the form of a graph (start_node,next_node).
def path_graph(node_value):
    graph=[]
    for n in range(len(node_value)-1):
        graph.append([node_value[n],node_value[n+1]])
    return graph

In [ ]:
# All significant (above-threshold) nodes were assigned a weighting that reflected
# their topological strength (i.e., the number of significant downstream nodes that are pointed to, either
# directly or via other significant nodes).
# An initiating child node, n_i, was ignored if non-significant, and the algorithm proceeds to the next child.
# Otherwise, we increase the weight counter by 1 and look for children of this node
# and so on. All non-significant nodes have NW = 0.

def sign_children(graph,weight=0):
    threshold=1
    if(len(graph)==0): # Reached the end
        return weight
    if(graph[0]<threshold):
        return sign_children(graph[1:],weight)
    else:
        return sign_children(graph[1:],weight+1) # Increase weight, if the current node/child is significant

def path_node_weight(node_value):
    node_weight=[]
    cur_weight=0
    for node in range(len(node_value)):
        cur_weight=sign_children(node_value[node:])
        node_weight.append(cur_weight)
    return node_weight

#### 1.4. Node_score (NS)
NV and NW values are combined to calculate a Node_Score (NS)

In [ ]:
def path_node_score(NV,NW):
    node_score=[]
    for n in range(len(NV)):
        if(NV[n]>1):
            node_score.append(NV[n]*NW[n])
        else:
            node_score.append(0)
    return node_score

#### 1.5. PRS

In [ ]:
def path_PRS(log_fc,fc,pvalues):
    NV=path_node_value(log_fc,fc,pvalues)
    NW=path_node_weight(NV)
    NS=path_node_score(NV,NW)
    return sum(NS)